In [1]:
import numpy as np
import torch
import torch.nn as nn
from scipy import interpolate
import matplotlib.pyplot as plt
from tqdm import tqdm

from vanilla_fem import *
from pod_fem import *
from deim_fem import *
from neim_fem2 import *
from other_functions import *

## Parameters
The parameters which we will let vary are $a,b,c$ so that
$$
    (a, b, c) \in \mathbb{P} = [-0.5, 0.5]\times[-0.5, 0.5]\times[1,2].
$$

In [2]:
# model parameters
parameters = []
for a in np.linspace(-0.5, 0.5, 2):
    for b in np.linspace(-0.5, 0.5, 2):
        for c in np.linspace(1, 2, 2):
            parameters.append((a, b, c))

A0 = 500
M = 1
L = 0.1

# number of time steps and final time
Nt = 200
t_final = 5
tVals = np.linspace(0, t_final, Nt)
dt = tVals[1] - tVals[0] # delta t

In [3]:
# define the mesh in space
mesh, interior_point_coords = get_triangulation(refinements=5)

# Get the finite element space, stiffness matrix, lumped mass matrix as a vector, and mass matrix
Vh, stiffness_matrix, gamma, mass_matrix = calculate_basis_integrals(mesh)

## Solve High Fidelity PDE
Note that we only keep track of nodes in the interior of $\Omega$ because we use homogeneous Dirichlet boundary conditions.

In [4]:
hf_solutions = []
for (a, b, c) in parameters:
    # Get vectors for the components of matrices (flattened over space) 
    # over time with time 0 initialized.
    Q1, Q2, p1, p2, r, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)

    # solve the PDE (updates the entries of Q1, Q2, etc. within the function)
    solve_Q_flow(Q1, Q2, p1, p2, r, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)
    
    hf_solutions.append((Q1, Q2, p1, p2, r))

Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 103.16it/s]


Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 106.57it/s]


Computing scheme...


...

100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 112.51it/s]


Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 111.20it/s]


Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 109.22it/s]


## Compute POD Matrices

In [5]:
Q1 = []
Q2 = []
p1 = []
p2 = []
r  = []
for sol in hf_solutions:
    Q1.append(sol[0])
    Q2.append(sol[1])
    p1.append(sol[2])
    p2.append(sol[3])
    r.append(sol[4])

Q1 = np.concatenate(Q1, axis=0)
Q2 = np.concatenate(Q2, axis=0)
p1 = np.concatenate(p1, axis=0)
p2 = np.concatenate(p2, axis=0)
r  = np.concatenate(r, axis=0)

max_rank = 5
U_Q1, U_Q2, U_r = get_POD_matrices(Q1, Q2, r, max_rank)

## Compare POD Solution to High Fidelity Solution (New Parameter)

In [6]:
a, b, c = -0.2, 0.2, 1.5

Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

Q1_pod, Q2_pod, p1_pod, p2_pod, r_pod = initialize_Q_flow_POD(Q1_, Q2_, p1_, p2_, r_, U_Q1, U_Q2, U_r)
solve_Q_flow_POD(Q1_pod, Q2_pod, p1_pod, p2_pod, r_pod, gamma, stiffness_matrix, U_Q1, U_Q2, U_r, Nt, a, b, c, A0, M, L, dt)

print("Relative Error:", np.linalg.norm(Q1_ - (U_Q1 @ Q1_pod.T).T) / np.linalg.norm(Q1_))

Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 105.99it/s]


Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:00<00:00, 790.00it/s]


Relative Error: 0.017428325319199647


## Compute DEIM Operators

In [7]:
deim_modes = 7

nonlinearityQ = M * gamma.reshape(-1, 1) * np.concatenate([p1 * r, p2 * r], axis=0).T
UN_Q, nQ_indices = get_DEIM_operators(nonlinearityQ, deim_modes)
U_deimQ1 = U_Q1.T @ UN_Q
U_deimQ2 = U_Q2.T @ UN_Q

nonlinearityR = (2 * p1[:-1] * (Q1[1:] - Q1[:-1]) + 2 * p2[:-1] * (Q2[1:] - Q2[:-1])).T
UN_R, nR_indices = get_DEIM_operators(nonlinearityR, deim_modes)
U_deimR = U_r.T @ UN_R

## Compare DEIM Solution to High Fidelity Solution (New Parameter)

In [8]:
a, b, c = 0, 0, 1.5

Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim = initialize_Q_flow_DEIM(Q1_, Q2_, p1_, p2_, r_, 
                                                                                          U_Q1, U_Q2, U_r, 
                                                                                          nQ_indices, nR_indices)
solve_Q_flow_DEIM(Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim, 
                    gamma, stiffness_matrix, 
                    U_Q1, U_Q2, U_r, U_deimQ1, U_deimQ2, U_deimR,
                    nQ_indices, nR_indices,
                    Nt, a, b, c, A0, M, L, dt)

print("Relative Error:", np.linalg.norm(Q1_ - (U_Q1 @ Q1_deim.T).T) / np.linalg.norm(Q1_))

Computing scheme...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:01<00:00, 106.55it/s]


Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:00<00:00, 3082.75it/s]


Relative Error: 0.017398615797489164


## Compute NEIM Operators

### $Q$ Equation NEIM

In [9]:
time_frequency = 10
mu = []
for (a, b, c) in parameters:
    for t in range(0, Nt, time_frequency):
        mu.append([t, a, b, c])
mu = np.array(mu)

num_time_params = len(range(0, Nt-1, time_frequency))

In [10]:
ro_sols = np.zeros((mu.shape[0], 3*max_rank))
f_NEIM  = np.zeros((mu.shape[0], mu.shape[0], 2*max_rank))

# iterate over solution parameter
for i, param1 in enumerate(mu):
    Q1_, Q2_, _, _, r_ = hf_solutions[i // num_time_params]
    Q1_, Q2_, r_ = Q1_[i % num_time_params], Q2_[i % num_time_params], r_[i % num_time_params]
    
    ro_sols[i] = np.concatenate([Q1_ @ U_Q1, Q2_ @ U_Q2, r_ @ U_r])
    
    # iterate over nonlinearity parameter
    for j, param2 in enumerate(mu):
        # evaluate Nonlinearity(solution(param1); param2)
        (t, a, b, c) = param2
        
        Q = np.concatenate((
                np.concatenate((Q1_[:, None, None], Q2_[:, None, None]), axis=2),
                np.concatenate((Q2_[:, None, None], -Q1_[:, None, None]), axis=2),
        ), axis=1)
        PQ = P(Q, a, b, c, A0)
        p1_ = PQ[:, 0, 0]
        p2_ = PQ[:, 0, 1]
        
        nonlinearityQ = M * np.concatenate([(gamma[None] * p1_ * r_) @ U_Q1, (gamma[None] * p2_ * r_) @ U_Q2], axis=1)
        f_NEIM[i, j] = nonlinearityQ.reshape(-1)

#ro_sols = (ro_sols - np.mean(ro_sols, axis=(0,1), keepdims=True)) / np.std(ro_sols, axis=(0,1), keepdims=True)

In [11]:
NQ_NEIM, _, _, _, _ = NEIM_Q(ro_sols, f_NEIM, mu, max_modes=3, train_loop_iterations=3000)

100 Max Error: 0.034939279515320686 Mean Error: 0.0015995184296879448
0 0.0011506466989221692
100 1.3085581169637271e-08
200 1.1749984083660281e-10
300 1.1715551518157727e-10
400 1.1715551439374466e-10
500 1.1715551437051384e-10
600 1.1715551437051384e-10
700 1.1715551437051384e-10
800 1.1715551437051384e-10
900 1.1715551437051384e-10
1000 1.1715551437051384e-10
1100 1.1715551437051384e-10
1200 1.1715551437051384e-10
1300 1.1715551437051384e-10
1400 1.1715551437051384e-10
1500 1.1715551437051384e-10
1600 1.1715551437051384e-10
1700 1.1715551437051384e-10
1800 1.1715551437051384e-10
1900 1.1715551437051384e-10
2000 1.1715551437051384e-10
2100 1.1715551437051384e-10
2200 1.1715551437051384e-10
2300 1.1715551437051384e-10
2400 1.1715551437051384e-10
2500 1.1715551437051384e-10
2600 1.1715551437051384e-10
2700 1.1715551437051384e-10
2800 1.1715551437051384e-10
2900 1.1715551437051384e-10
Mean Already Selected Error: 6.500142829882783e-11
141 Max Error: 0.00127589036927408 Mean Error: 0.000

In [12]:
mu[100]

array([ 0. ,  0.5, -0.5,  2. ])

In [13]:
(np.linalg.norm(NQ_NEIM(np.array([0.0, 0.5, -0.5, 2]), ro_sols[[100]]) - f_NEIM[100, 100]) / np.linalg.norm(f_NEIM[100, 100]),
  np.linalg.norm(NQ_NEIM(np.array([0.0, 0.5, -0.5, 2]), ro_sols[[100]]) - f_NEIM[100, 100]))

(4.2969644858642865e-05, 8.031908213867319e-06)

In [14]:
f_NEIM[100, 100], NQ_NEIM(np.array([0.0, 0.5, -0.5, 2]), ro_sols[[100]])

(array([-0.09278623, -0.06521712, -0.03681794,  0.02702244,  0.00824505,
        -0.12329852, -0.02897234, -0.04548568, -0.02107722,  0.03698445]),
 array([-0.09278246, -0.06521428, -0.03681624,  0.02702122,  0.00824467,
        -0.12329334, -0.02897105, -0.04548348, -0.0210762 ,  0.03698262],
       dtype=float32))

### $r$ Equation NEIM

In [15]:
time_frequency = 10
mu = []
for (a, b, c) in parameters:
    for t in range(0, Nt-1, time_frequency):
        mu.append([t, a, b, c])
mu = np.array(mu)

num_time_params = len(range(0, Nt-1, time_frequency))

In [16]:
ro_sols = np.zeros((mu.shape[0], 4*max_rank))
f_NEIM  = np.zeros((mu.shape[0], mu.shape[0], max_rank))

# iterate over solution parameter
for i, param1 in enumerate(mu):
    Q1_, Q2_, _, _, r_ = hf_solutions[i // num_time_params]
    time_idx = i % num_time_params
    
    Q1_0, Q2_0, r_0 = Q1_[time_idx], Q2_[time_idx], r_[time_idx]
    Q1_1, Q2_1, r_1 = Q1_[time_idx+1], Q2_[time_idx+1], r_[time_idx+1]
    
    ro_sols[i] = np.concatenate([Q1_1 @ U_Q1, Q1_0 @ U_Q1, Q2_1 @ U_Q2, Q2_0 @ U_Q2])
    
    # iterate over nonlinearity parameter
    for j, param2 in enumerate(mu):
        # evaluate Nonlinearity(solution(param1); param2)
        (t, a, b, c) = param2
        
        Q = np.concatenate((
                np.concatenate((Q1_0[:, None, None], Q2_0[:, None, None]), axis=2),
                np.concatenate((Q2_0[:, None, None], -Q1_0[:, None, None]), axis=2),
        ), axis=1)
        PQ = P(Q, a, b, c, A0)
        p1_ = PQ[:, 0, 0]
        p2_ = PQ[:, 0, 1]
        
        nonlinearityR = (2 * p1_ * (Q1_1 - Q1_0) + 2 * p2_ * (Q2_1 - Q2_0)) @ U_r
        f_NEIM[i, j] = nonlinearityR.reshape(-1)

#ro_sols = (ro_sols - np.mean(ro_sols, axis=(0,1), keepdims=True)) / np.std(ro_sols, axis=(0,1), keepdims=True)

In [17]:
NR_NEIM, _, _, _, _ = NEIM_R(ro_sols, f_NEIM, mu, max_modes=3, train_loop_iterations=3000)

100 Max Error: 0.6887720783130706 Mean Error: 0.017199202483429876
0 0.0018829615268406655
100 4.6572966387292526e-08
200 2.5670705200649328e-08
300 2.478873977981283e-08
400 2.3949146999044734e-08
500 2.316230708266722e-08
600 2.2422989883486037e-08
700 2.1722352241777383e-08
800 2.1052126406200016e-08
900 2.040568691778705e-08
1000 1.9778054158505538e-08
1100 1.9165542929703883e-08
1200 1.8565456906071942e-08
1300 1.797594759320119e-08
1400 1.7395596089657917e-08
1500 1.682343755644078e-08
1600 1.6258715464883917e-08
1700 1.570097442419294e-08
1800 1.514981867496377e-08
1900 1.4604961490065678e-08
2000 1.406617480548038e-08
2100 1.353327806135957e-08
2200 1.3006111256385e-08
2300 1.2484621799003809e-08
2400 1.1968739826769807e-08
2500 1.1458518988750927e-08
2600 1.0954084114514194e-08
2700 1.0455686939544566e-08
2800 9.963745570818268e-09
2900 9.478801476564146e-09
Mean Already Selected Error: 4.153312159294982e-10
101 Max Error: 0.024827026768983896 Mean Error: 0.0009453568209928132

In [18]:
mu[100]

array([ 0. ,  0.5, -0.5,  2. ])

In [24]:
(np.linalg.norm(NR_NEIM(np.array([0.0, 0.5, -0.5, 2]), ro_sols[100]) - f_NEIM[100, 100]) / np.linalg.norm(f_NEIM[100, 100]),
  np.linalg.norm(NR_NEIM(np.array([0.0, 0.5, -0.5, 2]), ro_sols[100]) - f_NEIM[100, 100]))

(2.4904423802021347e-05, 2.0668752500369372e-05)

In [25]:
f_NEIM[100, 100], NR_NEIM(np.array([0.0, 0.5, -0.5, 2]), ro_sols[100])

(array([ 0.26432445,  0.78637389, -0.02225876, -0.00491231,  0.0010932 ]),
 array([ 0.26430967,  0.7863608 , -0.02226445, -0.00491462,  0.00109301],
       dtype=float32))